In [1]:
import json
import openai

In [2]:
training_data = []
validation_data = []

with open('emoji.json', 'r', encoding='utf-8') as emojis: 
    emojis = json.load(emojis)

training_size = int(len(emojis) * (2/3))

for emoji in range(len(emojis)):
    conversation = {}
    conversation["prompt"] = "Show the emoji with \"" + emojis[emoji]["description"] + "\" under category \"" + emojis[emoji]["category"] + "\""
    conversation["completion"] = emojis[emoji]["emoji"]
    if emoji <= training_size:
        training_data.append(conversation)
    else:
        validation_data.append(conversation)

In [3]:
print(len(validation_data), len(training_data))
print(training_data[0:3])

623 1247
[{'prompt': 'Show the emoji with "grinning face" under category "Smileys & Emotion"', 'completion': '😀'}, {'prompt': 'Show the emoji with "grinning face with big eyes" under category "Smileys & Emotion"', 'completion': '😃'}, {'prompt': 'Show the emoji with "grinning face with smiling eyes" under category "Smileys & Emotion"', 'completion': '😄'}]


In [4]:
training_file_name = "training_data.jsonl"
validation_file_name = "validation_data.jsonl"

In [5]:
def prepare_data(dictionary_data, file_name):
    with open(file_name, 'w', encoding='utf-8') as outfile:
        for entry in dictionary_data:
            json.dump(entry, outfile)
            outfile.write('\n')

In [6]:
prepare_data(training_data, training_file_name)
prepare_data(validation_data, validation_file_name)

In [ ]:
openai.api_key = ""

In [14]:
openai.File.create(
  file=open(training_file_name, "rb"),
  purpose="fine-tune"
)

<File file id=file-3OeP1zB7qNnQdV68lLtBIZGK at 0x2081e52eb10> JSON: {
  "object": "file",
  "id": "file-3OeP1zB7qNnQdV68lLtBIZGK",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 149151,
  "created_at": 1703149155,
  "status": "uploaded",
  "status_details": null
}

In [15]:
openai.File.create(
  file=open(validation_file_name, "rb"),
  purpose="fine-tune"
)

<File file id=file-fQIRoNzPUOE518Fu9jzlVKlx at 0x2081c79e990> JSON: {
  "object": "file",
  "id": "file-fQIRoNzPUOE518Fu9jzlVKlx",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 72271,
  "created_at": 1703149176,
  "status": "uploaded",
  "status_details": null
}

In [17]:
create_args = {
	"training_file": "file-3OeP1zB7qNnQdV68lLtBIZGK",
	"validation_file": "file-fQIRoNzPUOE518Fu9jzlVKlx",
	"model": "davinci",
	"n_epochs": 15,
	"batch_size": 3,
	"learning_rate_multiplier": 0.3
}

response = openai.FineTune.create(**create_args)
job_id = response["id"]
status = response["status"]

print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {status}")

Fine-tunning model with jobID: ft-hnIGWQR0qSeEsNqxMDCZyCTt.
Training Response: {
  "object": "fine-tune",
  "id": "ft-hnIGWQR0qSeEsNqxMDCZyCTt",
  "hyperparams": {
    "n_epochs": 15,
    "batch_size": 3,
    "prompt_loss_weight": 0.01,
    "learning_rate_multiplier": 0.3
  },
  "organization_id": "org-zztDjpGTnNuPt4Dwps364a8H",
  "model": "davinci",
  "training_files": [
    {
      "object": "file",
      "id": "file-3OeP1zB7qNnQdV68lLtBIZGK",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 149151,
      "created_at": 1703149155,
      "status": "processed",
      "status_details": null
    }
  ],
  "validation_files": [
    {
      "object": "file",
      "id": "file-fQIRoNzPUOE518Fu9jzlVKlx",
      "purpose": "fine-tune",
      "filename": "file",
      "bytes": 72271,
      "created_at": 1703149176,
      "status": "processed",
      "status_details": null
    }
  ],
  "result_files": [],
  "created_at": 1703149460,
  "updated_at": 1703149460,
  "status": "p